Here we will be implementing a version of CoCa classifier (the current top performer on ImageNet database) by training it on imagenet imstances of our classes and then asking it to classify our 'strange' images.

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files

files.upload(); # upload your kaggle.json file

import json

!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle/’: File exists
- path is now set to: {/content}


In [ ]:
#be careful, the data is like 160gb +
#!kaggle competitions download -c imagenet-object-localization-challenge

#CATS AND DOGS
!kaggle competitions download -c dogs-vs-cats #DATA FOR DOGS VS CATS

#HANDS:
downloaded = drive.CreateFile({'id':'1KcMYcNJgtK1zZvfl_9sTqnyBUTri2aP2'}) 
downloaded.GetContentFile('Hands.zip')

In [ ]:
!unzip dogs-vs-cats.zip
!unzip Hands.zip
#TODO: fcombine into same directory

!ls

In [ ]:
!pip install coca-pytorch
!pip install vit-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 627 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 5.4 MB/s 


In [ ]:
import torch

# import vision transformer

from vit_pytorch import ViT
from vit_pytorch.extractor import Extractor

vit = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

# extractor will enable it so the vision transformer returns its embeddings

# import CoCa and instantiate it

from coca_pytorch.coca_pytorch import CoCa

#FIXED: must enable GPU hardware acceleration in runtime settings or this will not work
coca = CoCa(
    dim = 512,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 1024,              # image embedding dimension, if not the same as model dimensions
    num_tokens = 20000,            # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

#TODO: replace these with imageNet instances

text = torch.randint(0, 20000, (4, 512)).cuda()
images = torch.randn(4, 3, 256, 256).cuda()

# train by giving CoCa your text and images with `return_loss = True`

loss = coca(
    text = text,
    images = images,
    return_loss = True  # set this to True to get the full caption + contrastive loss
)

loss.backward()

# TODO ^ train above on imagenet data

logits = coca(
    text = text,
    images = images
) # (4, 512, 20000)

# and the CLIP-like text and image embeddings as

text_embeds, image_embeds = coca(
    text = text,
    images = images,
    return_embeddings = True
) # (4, 512), (4, 512)

CITATIONS/REFERENCES:

@article{afifi201911kHands, 
title = {11K Hands: gender recognition and biometric identification using a large dataset of hand images}, author = {Afifi, Mahmoud}, journal = {Multimedia Tools and Applications}, doi = {10.1007/s11042-019-7424-8}, url = {https://doi.org/10.1007/s11042-019-7424-8}, year={2019} 

@inproceedings{Yu2022CoCaCC,
  title   = {CoCa: Contrastive Captioners are Image-Text Foundation Models},
  author  = {Jiahui Yu and Zirui Wang and Vijay Vasudevan and Legg Yeung and Mojtaba Seyedhosseini and Yonghui Wu},
  year    = {2022}
}}